In [1]:

import numpy
import pandas 
import keras
import tensorflow.contrib.eager as tfe
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.utils import shuffle

Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
seed = 7
numpy.random.seed(seed)

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [24]:
X.shape[1]

60

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
reverse_encoding = encoder.inverse_transform(encoded_Y) # reversing the encoding for fun


In [26]:
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
def create_baseline():
  model = models.Sequential()
  model.add(layers.Dense(60,activation='relu',input_shape=(X.shape[1],)))
  model.add(layers.Dense(1,activation='sigmoid'))
  model.compile(optimizer='Adam', loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [0]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)

In [29]:
results

array([0.85714287, 0.90476191, 0.80952382, 0.71428572, 0.95238096,
       0.80952381, 0.71428572, 0.80952382, 0.85000001, 0.70000001])

In [30]:
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 81.21% (7.97%)


In [34]:
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
# print(estimators)
estimators.append(('mlp',KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5,verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline,X,encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 83.64% (7.94%)


In [59]:
pre =  X[120:121]
ans = encoded_Y[120]
print(pre,ans)

[[0.0346 0.0509 0.0079 0.0243 0.0432 0.0735 0.0938 0.1134 0.1228 0.1508
  0.1809 0.239  0.2947 0.2866 0.401  0.5325 0.5486 0.5823 0.6041 0.6749
  0.7084 0.789  0.9284 0.9781 0.9738 1.     0.9702 0.9956 0.8235 0.602
  0.5342 0.4867 0.3526 0.1566 0.0946 0.1613 0.2824 0.339  0.3019 0.2945
  0.2978 0.2676 0.2055 0.2069 0.1625 0.1216 0.1013 0.0744 0.0386 0.005
  0.0146 0.004  0.0122 0.0107 0.0112 0.0102 0.0052 0.0024 0.0079 0.0031]] 0


In [44]:
pipeline.fit(X,encoded_Y)

Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('mlp',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f92acd17a20>)],
         verbose=False)

In [60]:
pipeline.predict(pre)

array([[0]])